<a href="https://colab.research.google.com/github/NikhilHooda/EPL_Web_Scrapper/blob/main/EPL_Web_Scrapper.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import requests

In [ ]:
standings_url = "https://fbref.com/en/comps/9/Premier-League-Stats"

In [ ]:
data = requests.get(standings_url) ## get html code

In [ ]:
from bs4 import BeautifulSoup

In [ ]:
soup = BeautifulSoup(data.text) ## parse html code

In [ ]:
standings_table = soup.select('table.stats_table')[0] ## parse html code for first team in EPL stats table

In [ ]:
links = standings_table.find_all('a') ## all team links are a tags

In [ ]:
links = [l.get("href") for l in links] ## make links a list of href links

In [ ]:
links = [l for l in links if '/squads/' in l] ## filter links for only squad links

In [ ]:
team_urls = [f"https://fbref.com{l}" for l in links] ## add on the start part of link to make it a full link

In [ ]:
team_url = team_urls[0] ## access first teams link

In [ ]:
data = requests.get(team_url)

In [ ]:
data.text

In [ ]:
import pandas as pd

matches = pd.read_html(data.text, match="Scores & Fixtures")

In [ ]:
matches[0]

In [ ]:
soup = BeautifulSoup(data.text)

In [ ]:
links = soup.find_all('a')

In [ ]:
links = [l.get("href") for l in links]

In [ ]:
links = [l for l in links if l and 'all_comps/shooting/' in l]

In [ ]:
links

In [ ]:
data = requests.get(f"https://fbref.com{links[0]}")

In [ ]:
shooting = pd.read_html(data.text, match="Shooting")[0]

In [ ]:
shooting.head()

,Date,Time,Comp,Round,Day,Venue,Result,GF,GA,Opponent,...,Dist,FK,PK,PKatt,xG,npxG,npxG/Sh,G-xG,np:G-xG,Match Report
0,2022-08-06,15:00,Premier League,Matchweek 1,Sat,Away,L,0.0,2.0,Bournemouth,...,24.6,0.0,0,0,0.7,0.7,0.04,-0.7,-0.7,Match Report
1,2022-08-13,12:30,Premier League,Matchweek 2,Sat,Home,W,2.0,1.0,Everton,...,13.3,0.0,0,0,2.3,2.3,0.19,-0.3,-0.3,Match Report
2,2022-08-20,15:00,Premier League,Matchweek 3,Sat,Away,L,1.0,3.0,Crystal Palace,...,17.8,0.0,0,0,0.9,0.9,0.07,0.1,0.1,Match Report
3,2022-08-23,19:45,EFL Cup,Second round,Tue,Away,W,4.0,1.0,Bolton,...,NaN,NaN,1,1,NaN,NaN,NaN,NaN,NaN,Match Report
4,2022-08-28,14:00,Premier League,Matchweek 4,Sun,Home,L,0.0,1.0,West Ham,...,20.5,0.0,0,0,0.5,0.5,0.05,-0.5,-0.5,Match Report


In [ ]:
shooting.columns = shooting.columns.droplevel() ## to drop the top bolded header row bc we dont need

In [ ]:
team_data = matches[0].merge(shooting[["Date", "Sh", "SoT", "Dist", "FK", "PK", "PKatt"]], on="Date") ## merge team and shooting data together into one dataframe

In [ ]:
matches[0].shape

In [ ]:
years = list(range(2024, 2020, -1))

In [ ]:
all_matches = []

In [139]:
standings_url = "https://fbref.com/en/comps/9/Premier-League-Stats"

In [ ]:
import time
for year in years:
  data = requests.get(standings_url)
  soup = BeautifulSoup(data.text)
  standings_table = soup.select('table.stats_table')[0]

  links = [l.get("href") for l in standings_table.find_all('a')]
  links = [l for l in links if '/squads/' in l]
  team_urls = [f"https://fbref.com{l}" for l in links]

  previous_season = soup.select("a.prev")[0].get("href")
  standings_url = f"https://fbref.com{previous_season}"

  for team_url in team_urls:
    team_name = team_url.split("/")[-1].replace("-Stats", "").replace("-", " ")

    data = requests.get(team_url)
    matches = pd.read_html(data.text, match="Scores & Fixtures")[0]

    soup = BeautifulSoup(data.text)
    links = [l.get("href") for l in soup.find_all('a')]
    links = [l for l in links if l and 'all_comps/shooting/' in l]
    data = requests.get(f"https://fbref.com{links[0]}")
    shooting = pd.read_html(data.text, match="Shooting")[0]
    shooting.columns = shooting.columns.droplevel()

    try:
      team_data = matches.merge(shooting[["Date", "Sh", "SoT", "Dist", "FK", "PK", "PKatt"]], on="Date")
    except ValueError:
      continue

    team_data = team_data[team_data["Comp"] == "Premier League"]
    team_data["Season"] = year
    team_data["Team"] = team_name
    all_matches.append(team_data)
    time.sleep(1) ## prevent delays from too many requests too

In [ ]:
match_df = pd.concat(all_matches)

In [ ]:
match_df

In [ ]:
match_df.columns = [c.lower() for c in match_df.columns]

In [ ]:
match_df.to_csv("matches.csv")